<a href="https://colab.research.google.com/github/AhmedSamirScience/Prediction-of-adaptor-proteins-using-PSSM-profiles-and-Recurrent-Neural-Networks.-/blob/main/PredictionOfAdaptorAndNonAdaptorProteinFinalVersion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import numpy as np
from torch import nn
from torch.autograd import Variable
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F
from google.colab import drive
import pickle
import time
import math
import os
import csv
import glob
from sklearn import metrics
from sklearn.model_selection import StratifiedKFold


In [ ]:
drive.mount('/content/gdrive/')
path = "gdrive/MyDrive/proteinsproject"
os.chdir(path)

Mounted at /content/gdrive/


In [ ]:
!pwd

/content/gdrive/MyDrive/proteinsproject


In [ ]:
split_seed = 53
MODEL_DIR = 'model'
if not os.path.exists(MODEL_DIR):
    os.makedirs(MODEL_DIR)
            
file_model = "adaptor.model"

CONV1D_FEATURE_SIZE = 256 # So bo loc
CONV1D_KERNEL_SIZE = 3 # Kich thuoc bo loc
AVGPOOL1D_KERNEL_SIZE = 3 # Kich thuoc bo loc trung binh
GRU_HIDDEN_SIZE = 256
FULLY_CONNECTED_LAYER_SIZE = 512

NUMBER_EPOCHS = 30
LEARNING_RATE = 0.0001

LOSS_WEIGHT_POSITIVE = 10.07
LOSS_WEIGHT_NEGATIVE = 1.11

In [ ]:
def load_text_file(file_text):
    start_time = time.time()
    with open(file_text) as f:
        lines = f.readlines()
    return lines

def load_lst_file(file_lst, dir_name):
    lst_file = load_text_file(file_lst)
    lst_path = [dir_name + file_name.strip() for file_name in lst_file]
    return lst_path

In [ ]:
def create_list_train():
    lst_path_positive_train = glob.glob("./datasettrain/positive/*.pssm") 
    lst_path_negative_train = glob.glob("./datasettrain/negative/*.pssm")     
    
    print("Positive train: ", len(lst_path_positive_train))
    print("Negative train: ", len(lst_path_negative_train))          
    
    lst_positive_train_label = [1] *  len(lst_path_positive_train)
    lst_negative_train_label = [0] *  len(lst_path_negative_train)
    
    lst_path_train = lst_path_positive_train + lst_path_negative_train
    lst_label_train = lst_positive_train_label + lst_negative_train_label
    
    print("Train all: ", len(lst_path_train))
    print("")
    return lst_path_train, lst_label_train

In [ ]:
class BioinformaticsDataset(Dataset):
    # X: list cac file (full path)
    # Y: list label [0, 1]; 0: negative, 1: positive
    def __init__(self, X, Y):
        self.X = X
        self.Y = Y

    def __getitem__(self, index):
        label = self.Y[index]
        lines = load_text_file(self.X[index])
        start_line = 3
        end_line = len(lines) - 7
        values = np.zeros((end_line - start_line + 1, 20))

        for i in range(start_line, end_line + 1):
            #print i
            strs = lines[i].strip().split()[2:22]
            #print strs
            for j in range(20):
                values[i-start_line][j] = int(strs[j])
                #print(values[i-start_line][j])
            #break

        #print("values: ", values)
        input = torch.from_numpy(values)
        return input, label

    def __len__(self):
        #return 100
        return len(self.X)


In [ ]:

def weighted_binary_cross_entropy(output, target, weights=True):        
    if weights :        
        loss = LOSS_WEIGHT_POSITIVE * (target * torch.log(output)) + \
               LOSS_WEIGHT_NEGATIVE * ((1 - target) * torch.log(1 - output))
    else:
        loss = target * torch.log(output) + (1 - target) * torch.log(1 - output)

    return torch.neg(torch.mean(loss))
    
class RNNModel(nn.Module):
    def __init__(self, n_layers=1):
        super(RNNModel, self).__init__()
        self.c1 = nn.Conv1d(20, CONV1D_FEATURE_SIZE, CONV1D_KERNEL_SIZE)
        self.p1 = nn.AvgPool1d(AVGPOOL1D_KERNEL_SIZE)
        self.c2 = nn.Conv1d(CONV1D_FEATURE_SIZE, CONV1D_FEATURE_SIZE, CONV1D_KERNEL_SIZE)
        self.p2 = nn.AvgPool1d(AVGPOOL1D_KERNEL_SIZE)
        self.gru = nn.GRU(CONV1D_FEATURE_SIZE, GRU_HIDDEN_SIZE, n_layers, dropout=0.01)
        self.fc = nn.Linear(GRU_HIDDEN_SIZE, FULLY_CONNECTED_LAYER_SIZE)
        self.fc_drop = nn.Dropout()
        self.out = nn.Linear(FULLY_CONNECTED_LAYER_SIZE, 1)
        self.out_act = nn.Sigmoid()

        self.gru_layers = n_layers
        
        self.criterion = nn.BCELoss()
        self.optimizer = torch.optim.SGD(self.parameters(), lr=LEARNING_RATE)

    def forward(self, inputs):
        batch_size = inputs.size(0)
        #print "inputs size: ", inputs.size()
        #print "batch_size: ", batch_size
        #######################
        #  USE GPU FOR MODEL  #
        #######################
        if torch.cuda.is_available():
            h0 = Variable(torch.zeros(self.gru_layers, inputs.size(0), GRU_HIDDEN_SIZE).cuda())
        else:
            h0 = Variable(torch.zeros(self.gru_layers, inputs.size(0), GRU_HIDDEN_SIZE))
        #print h0.size()

        # Turn (batch_size x seq_len) into (batch_size x input_size x seq_len) for CNN
        inputs = inputs.transpose(1,2)
        #print "inputs size: ", inputs.size()
        c = self.c1(inputs)
        #print "c1 : ", c.size()
        p = self.p1(c)
        #print "p1: ", p.size()
        c = self.c2(p)
        #print "c2: ", c.size()
        p = self.p2(c)
        #print "p2: ", p.size()

        # Turn (batch_size x hidden_size x seq_len) back into (seq_len x batch_size x hidden_size) for RNN
        p = p.transpose(1, 2).transpose(0, 1)

        #p = F.tanh(p)
        p = F.relu(p)
        #print "p: ", p.size()

        #output, hidden = self.gru(p, hidden)
        output, hidden = self.gru(p, h0)
        #conv_seq_len = output.size(0)
        #print "gru output: ", output.size()
        #print "conv_seq_len: ", conv_seq_len
        #print "batch_size: ", batch_size
        #print "GRU_HIDDEN_SIZE: ", GRU_HIDDEN_SIZE

        #output = output.view(conv_seq_len * batch_size, GRU_HIDDEN_SIZE)
        #print "last gru output: ", output.size()
        #output = torch.gather(output, 1, torch.tensor(30))
        #print "last gru output: ", output.size()

        #output = F.tanh(self.out(output))
        #print("Hidden: ", hidden.size())

        output = F.relu(self.fc(hidden))
        #print("output fc: ", output.size())
        output = self.out(output)
        #print("output out: ", output.size())
        output = self.out_act(output)
        #print("output sigmoid: ", output.size())
        #output = output.view(conv_seq_len, -1, NUMBER_CLASSES)

        return output
        
    def train_one_epoch(self, learning_rate):
        self.train()
        for param_group in self.optimizer.param_groups:
            param_group['lr'] = learning_rate

        epoch_loss_train = 0.0
        nb_train = 0
        nb_train_short = 0
        for i, data in enumerate(self.train_loader, 0):
            inputs, labels = data
            inputs_length = inputs.size()[1]
            if inputs_length < 20:
                nb_train_short += 1
                continue

            #continue

            inputs = inputs.float()
            labels = labels.float()
            
            if torch.cuda.is_available():
                inputs = Variable(inputs.cuda())
                labels = Variable(labels.cuda())

            else:
                inputs = Variable(inputs)
                labels = Variable(labels)

            self.optimizer.zero_grad()

            # pass to get output/logits
            outputs = self(inputs)
            outputs = outputs[-1,-1]
            loss = weighted_binary_cross_entropy(outputs, labels)
            #print("loss: ", loss.data[0])
            #loss = torch.mul(loss, weight)

            # Getting gradients w.r.t. parameters
            loss.backward()

            # Updating parameters
            self.optimizer.step()

            epoch_loss_train = epoch_loss_train + loss.item()
            nb_train = nb_train + 1

            #if i > 20: break
            #if i % 10 == 0:
            #    print("Loss: ", i, " : ", epoch_loss_train/nb_train)

        print("nb train short (length < 20): ", nb_train_short)

        epoch_loss_train_avg = epoch_loss_train / nb_train
        return epoch_loss_train_avg
        
    def train_validate(self, train_dataset, validate_dataset, fold):
        self.train_loader = DataLoader(dataset=train_dataset, batch_size=1,                              
                              shuffle=True, num_workers=4)
        self.val_loader = DataLoader(dataset=validate_dataset, batch_size=1,                              
                              shuffle=True, num_workers=4)                                        
        
        #file_model = "adaptor_model_{}_{}.pfl".format(fold, best_epoch)
        file_model = "best_model_saved.pkl"
        
        train_losses = []
        train_lst_acc = []
        train_lst_sensitivity = []
        train_lst_specificity = []
        train_lst_mcc = []
        train_lst_auc = []
          
        val_losses = []
        val_lst_acc = []
        val_lst_sensitivity = []
        val_lst_specificity = []
        val_lst_mcc = []
        val_lst_auc = []

        best_val_loss = 1000
        best_epoch = 0
        for epoch in range(NUMBER_EPOCHS):
            print("\n############### EPOCH : ", epoch)
            epoch_loss_train_avg = self.train_one_epoch(LEARNING_RATE)
            train_losses.append(epoch_loss_train_avg)

            print("\nEvaluate Train set")
            result = self.evaluate(self.train_loader)
            print("\nTrain loss: ", result['epoch_loss_avg'])
            print("Train acc: ", result['acc'])
            print("Train confusion matrix: \n", result['confusion_matrix'])
            print("Train sensitivity: ", result['sensitivity'])
            print("Train specificity: ", result['specificity'])
            print("Train mcc: ", result['mcc'])
            print("Train auc: ", result['auc'])
            
            train_lst_acc.append(result['acc'])
            train_lst_sensitivity.append(result['sensitivity'])
            train_lst_specificity.append(result['specificity'])
            train_lst_mcc.append(result['mcc'])
            train_lst_auc.append(result['auc'])
            
            print("\nEvaluate Val set")
            result_val = self.evaluate(self.val_loader)
            print("\nVal loss: ", result_val['epoch_loss_avg'])
            print("Val acc: ", result_val['acc'])
            print("Val confusion matrix: \n", result_val['confusion_matrix'])
            print("Val sensitivity: ", result_val['sensitivity'])
            print("Val specificity: ", result_val['specificity'])
            print("Val mcc: ", result_val['mcc'])
            print("Val auc: ", result_val['auc'])
            
            val_losses.append(result_val['epoch_loss_avg'])
            val_lst_acc.append(result_val['acc'])
            val_lst_sensitivity.append(result_val['sensitivity'])
            val_lst_specificity.append(result_val['specificity'])
            val_lst_mcc.append(result_val['mcc'])
            val_lst_auc.append(result_val['auc'])
            
            if best_val_loss > result_val['epoch_loss_avg']:
                torch.save(self.state_dict(), os.path.join(MODEL_DIR, file_model))
                best_val_loss = result_val['epoch_loss_avg']
                print("Save model, best_val_loss: ", best_val_loss)
                best_epoch = epoch 

        print("\ntrain_losses: ", train_losses)
        print("train_lst_acc: ", train_lst_acc)
        print("train_lst_sensitivity: ", train_lst_sensitivity)
        print("train_lst_specificity: ", train_lst_specificity)
        print("train_lst_mcc: ", train_lst_mcc)
        print("train_lst_auc: ", train_lst_auc)
        
        print("\nval_losses: ", val_losses)
        print("val_lst_acc: ", val_lst_acc)
        print("val_lst_sensitivity: ", val_lst_sensitivity)
        print("val_lst_specificity: ", val_lst_specificity)
        print("val_lst_mcc: ", val_lst_mcc)
        print("val_lst_auc: ", val_lst_auc)
        
        #load again current_model_save for next training
        self.load_state_dict(torch.load(MODEL_DIR+"/best_model_saved.pkl"))
        #Luu lai model tot nhat ung voi fold validation hien tai
        model_fn = "adaptor_{}_{}.pkl".format(fold, best_epoch)
        torch.save(self.state_dict(), os.path.join(MODEL_DIR, model_fn))
        print(model_fn, "saved")
        
        #Save train_loss & val_loss
        with open(MODEL_DIR+"/logfile_loss_model_{}_{}.csv".format(fold, best_epoch), mode = 'w') as lf_loss:
            lf_loss = csv.writer(lf_loss, delimiter=',')
            lf_loss.writerow(['epoch', 'train loss', 'train acc', 'train_sens', 
                'train_spec', 'train_mcc', 'train_auc', 
                'val loss', 'val acc', 'val_sens', 'val_spec', 'val_mcc', 'val_auc'])
            for i in range(np.size(train_losses)):                
                lf_loss.writerow([i, train_losses[i], train_lst_acc[i],
                    train_lst_sensitivity[i], train_lst_specificity[i],
                    train_lst_mcc[i], train_lst_auc[i],
                    val_losses[i], val_lst_acc[i], val_lst_sensitivity[i],
                    val_lst_specificity[i], val_lst_mcc[i], val_lst_auc[i]])

    def evaluate(self, loader):
        self.eval()

        epoch_loss = 0.0
        nb = 0
        nb_short = 0

        arr_labels = []
        arr_labels_hyp = []
        arr_prob = []
        
        for i, data in enumerate(loader, 0):
            # get the inputs
            inputs, labels = data
            #print "labels: ", labels
            # print("inputs: ", inputs.size())
            inputs_length = inputs.size()[1]
            # print("inputs_length: ", inputs_length)
            if inputs_length < 20:
                nb_short += 1
                continue

            #print "labels: ", labels.cpu().numpy()[0]
            arr_labels.append(labels.cpu().numpy()[0])

            inputs = inputs.float()
            labels = labels.float()

            # print "\nBatch i : ", i
            # print "inputs: ", inputs
            # print "labels: ", labels

            #######################
            #  USE GPU FOR MODEL  #
            #######################
            if torch.cuda.is_available():
                inputs = Variable(inputs.cuda())
                labels = Variable(labels.cuda())
            else:
                inputs = Variable(inputs)
                labels = Variable(labels)

            # print(epoch, i, "inputs", inputs.data, "labels", labels.data)

            # Clear gradients w.r.t. parameters
            #optimizer.zero_grad()

            # Forward pass to get output/logits
            outputs = self(inputs)
            outputs = outputs[-1, -1]

            # loss = criterion(outputs[-1], labels)
            # print("outputs: ", outputs.size())
            # print("labels: ", labels.size())
            # print(outputs)
            # print(labels)
            #loss = criterion(outputs, labels)
            loss = weighted_binary_cross_entropy(outputs, labels)

            #print("outputs: ", outputs)
            arr_prob.append(outputs.data.cpu().numpy()[0])
            if outputs.data.cpu().numpy()[0] > 0.5: arr_labels_hyp.append(1)
            else: arr_labels_hyp.append(0)

            #epoch_loss = epoch_loss + loss.data[0]
            epoch_loss = epoch_loss + loss.item()
            nb = nb + 1

            #if i > 20: break
            #if i % 10 == 0:
            #    print("Loss: ", i, " : ", epoch_loss/nb)

        print("nb short (length < 20): ", nb_short)

        #print "arr_labels: ", arr_labels
        #print "arr_labels_hyp: ", arr_labels_hyp
        epoch_loss_avg = epoch_loss / nb
        
        auc = metrics.roc_auc_score(arr_labels, arr_prob)
        #print("auc: ", auc)

        acc, confusion_matrix, sensitivity, specificity, mcc = self.calculate_confusion_matrix(arr_labels, arr_labels_hyp)
        result = {'epoch_loss_avg': epoch_loss_avg, 
                    'acc' : acc, 
                    'confusion_matrix' : confusion_matrix,
                    'sensitivity' : sensitivity,
                    'specificity' : specificity,
                    'mcc' : mcc,
                    'auc' : auc,
                    'arr_prob': arr_prob,
                    'arr_labels': arr_labels
                     }
        return result
    
    def calculate_accuracy(self, arr_labels, arr_labels_hyp):
        corrects = 0

        for i in range(len(arr_labels)):
            if arr_labels[i] == arr_labels_hyp[i]:
                corrects = corrects + 1

        acc = corrects * 1.0 / len(arr_labels)
        return acc

    def calculate_confusion_matrix(self, arr_labels, arr_labels_hyp):
        corrects = 0
        confusion_matrix = np.zeros((2, 2))

        for i in range(len(arr_labels)):
            confusion_matrix[arr_labels_hyp[i]][arr_labels[i]] += 1

            if arr_labels[i] == arr_labels_hyp[i]:
                corrects = corrects + 1

        acc = corrects * 1.0 / len(arr_labels)
        specificity = confusion_matrix[0][0] / (confusion_matrix[0][0] + confusion_matrix[1][0])
        sensitivity = confusion_matrix[1][1] / (confusion_matrix[0][1] + confusion_matrix[1][1])
        tp = confusion_matrix[1][1]
        tn = confusion_matrix[0][0]
        fp = confusion_matrix[1][0]
        fn = confusion_matrix[0][1]
        mcc = (tp * tn - fp * fn ) / math.sqrt((tp + fp)*(tp + fn)*(tn + fp)*(tn + fn))
        #print("mcc: ", mcc)
        return acc, confusion_matrix, sensitivity, specificity, mcc
        
def training():
    print("split_seed: ", split_seed)
    skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=split_seed)
    #Split TRAIN_VAL dataset into 5-fold, 4 for training and 1 for cross validation
    fold = 0
    for train_index, val_index in skf.split(lst_path_train_all, lst_label_train_all):
        #get train and crossvalidation data by splitted indexes
        print("train_index: ", train_index)
        print("val_index: ", val_index)
        X_train = [lst_path_train_all[i] for i in train_index]
        X_val = [lst_path_train_all[i] for i in val_index]
        Y_train = [lst_label_train_all[i] for i in train_index]
        Y_val = [lst_label_train_all[i] for i in val_index]
        
        #print("X_train indices: ", train_index)
        #print("X_val indices: ", val_index)
        print("Y_train len: ", len(Y_train), "Y_val len:", len(Y_val))
        unique, count = np.unique(Y_train, return_counts = True)
        print("Y_train values: ", unique, "count: ", count)
        unique, count = np.unique(Y_val, return_counts = True)
        print("y_val values: ", unique, "count: ", count)
        print("\n")
        
        train_set = BioinformaticsDataset(X_train, Y_train)
        val_set = BioinformaticsDataset(X_val, Y_val)
        
        model = RNNModel(2)
        print(model)
        if torch.cuda.is_available():
            model.cuda()
            
        model.train_validate(train_set, val_set, fold)
        
        fold += 1
        # #break

    
############### MAIN PROGRAM ##############
if __name__ == "__main__":
    print("Adaptor Training K Fold\n")
    lst_path_train_all, lst_label_train_all = create_list_train()
        
    training()
    
    


In [ ]:
def create_list_test(): 
    lst_path_positive_test = glob.glob("./datasettest/positive/*.pssm") 
    lst_path_negative_test = glob.glob("./datasettest/negative/*.pssm")       
    
    print("Positive test: ", len(lst_path_positive_test))
    print("Negative test: ", len(lst_path_negative_test))
    
    lst_positive_test_label = [1] *  len(lst_path_positive_test)
    lst_negative_test_label = [0] *  len(lst_path_negative_test)
    lst_path_test_all = lst_path_positive_test + lst_path_negative_test
    lst_label_test_all = lst_positive_test_label + lst_negative_test_label
    
    print("Test all: ", len(lst_path_test_all))
    print("")
    return lst_path_test_all, lst_label_test_all

In [ ]:
def test():
    test_set = BioinformaticsDataset(lst_path_test_all, lst_label_test_all)
    test_loader = DataLoader(dataset=test_set, batch_size=1,                              
                              shuffle=True, num_workers=4)
    model = RNNModel(2)
    print(model)
    if torch.cuda.is_available():
        model.cuda()
    model.load_state_dict(torch.load(MODEL_DIR+"/best_model_saved.pkl"))
    result = model.evaluate(test_loader)
    print("\nTest loss: ", result['epoch_loss_avg'])
    print("Test acc: ", result['acc'])
    print("Test confusion matrix: \n", result['confusion_matrix'])
    print("Test sensitivity: ", result['sensitivity'])
    print("Test specificity: ", result['specificity'])
    print("Test mcc: ", result['mcc'])
    print("Test auc: ", result['auc'])

def test_all():
    test_set = BioinformaticsDataset(lst_path_test_all, lst_label_test_all)
    test_loader = DataLoader(dataset=test_set, batch_size=1,                              
                              shuffle=True, num_workers=4)
    model = RNNModel(2)
    print(model)
    if torch.cuda.is_available():
        model.cuda()
    
    testresult_fn = "/test_result.csv"
    with open(testresult_fn, mode='w') as outfile:
        outfile = csv.writer(outfile, delimiter=',')
        outfile.writerow(['model_fn', 'Accuracy', 'AUC', 'MCC', 
            'Sensitivity', 'Specificity'])
    list_model_fn = sorted(glob.glob(MODEL_DIR+"/adaptor_*.pkl"))
    
    y_prob_mtx = []
    arr_labels = []
    print("==========================TESTING RESULT================================")
    for model_fn in list_model_fn:
        print(model_fn)
        model.load_state_dict(torch.load(model_fn))
        result = model.evaluate(test_loader)
        arr_labels = result['arr_labels']
        print("\nTest loss: ", result['epoch_loss_avg'])
        print("Test acc: ", result['acc'])
        print("Test confusion matrix: \n", result['confusion_matrix'])
        print("Test sensitivity: ", result['sensitivity'])
        print("Test specificity: ", result['specificity'])
        print("Test mcc: ", result['mcc'])
        print("Test auc: ", result['auc'])
        
        with open(testresult_fn, mode='a') as outfile:
            outfile = csv.writer(outfile, delimiter=',')
            outfile.writerow([model_fn, result['acc'], result['auc'], 
                result['mcc'], result['sensitivity'], result['specificity']])
        
        y_prob_mtx.append(result['arr_prob'])
        y_prob_mtx.append(result['arr_prob'])
        y_prob_mtx.append(result['arr_prob'])
        y_prob_mtx.append(result['arr_prob'])
        y_prob_mtx.append(result['arr_prob'])
        break

    y_prob_mtx = np.array(y_prob_mtx)
    print("y_prob_mtx: ", y_prob_mtx.shape, "\n", y_prob_mtx)
    
    y_prob_mean = np.mean(y_prob_mtx, axis=0)    
    
    auc = metrics.roc_auc_score(arr_labels, y_prob_mean)
    print("auc: ", auc)
    
    arr_labels_hyp = []
    for prob in y_prob_mean:
        if prob > 0.5: arr_labels_hyp.append(1)
        else: arr_labels_hyp.append(0)

    acc, confusion_matrix, sensitivity, specificity, mcc = model.calculate_confusion_matrix(arr_labels, arr_labels_hyp)
        
    print("Results of Ensemble models: ")
    print("Test acc: ", acc)
    print("Test confusion matrix: \n", confusion_matrix)
    print("Test sensitivity: ", sensitivity)
    print("Test specificity: ", specificity)
    print("Test mcc: ", mcc)
    print("Test auc: ", auc)

In [ ]:
if __name__ == "__main__":
    print("Adaptor Training K Fold\n")    
    lst_path_test_all, lst_label_test_all = create_list_test()        
    
    #test()
    test_all()